In [1]:
import pandas_gbq
import pandas as pd

In [2]:
import requests
import os
import json

def shorten_url(long_url, api_token, alias=None, domain="tinyurl.com", tags=None):

    api_url = "https://api.tinyurl.com/create"

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
        "Accept": "application/json", # Good practice to specify expected response type
    }

    payload = {
        "url": long_url,
        "domain": domain,
    }
    if alias:
        payload["alias"] = alias
    if tags:
        payload["tags"] = tags if isinstance(tags, list) else [tags] # Ensure tags is a list

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        result = response.json()

        if result.get("code") == 0 and result.get("data") and result["data"].get("tiny_url"):
            return result["data"]["tiny_url"]
        else:
            errors = result.get("errors", ["Unknown error from TinyURL API."])
            print(f"TinyURL API Error: {'; '.join(errors)}")
            print(f"Full response: {result}")
            return None

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code}")
        try:
            # Try to parse error details from the response body if available
            error_details = e.response.json()
            print(f"Details: {error_details.get('errors', [e.response.text])}")
        except json.JSONDecodeError:
            print(f"Raw error response: {e.response.text}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to TinyURL: {e}")
        return None
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"Error parsing TinyURL response: {e}")
        print(f"Response text: {response.text if 'response' in locals() else 'No response object'}")
        return None

In [4]:
description = "Esta tabla detalla un conjunto exhaustivo de buenas prácticas y acciones específicas diseñadas para predios agrícolas, probablemente enfocados en la producción de ciruelas (como sugiere el dominio 'ciruelacertificada.cl'), como parte de un estándar de sostenibilidad o certificación. Cada registro identifica una práctica con un código, la clasifica por nivel de importancia (Fundamental, Básico, etc.) y le asigna puntos, la categoriza dentro de una dimensión (Ambiente, Calidad, Gestión, Social, Ética) y un tema específico (Agua, Suelo, Residuos, etc.), describe la acción concreta a realizar, especifica los medios de verificación necesarios para demostrar su cumplimiento y proporciona un enlace web para obtener información adicional o recursos relacionados con dicha práctica."

In [5]:
table_schema = [
  {
    "name": "n",
    "type": "INTEGER",
    "mode": "NULLABLE",
    "description": "Número de ítem o identificador secuencial."
  },
  {
    "name": "codigo",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Código alfanumérico único de la práctica."
  },
  {
    "name": "nivel",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Nivel de la práctica (Ej: Fundamental, Básico, Intermedio, Avanzado)."
  },
  {
    "name": "puntos",
    "type": "INTEGER",
    "mode": "NULLABLE",
    "description": "Puntuación asignada a la práctica."
  },
  {
    "name": "dimension",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Dimensión a la que pertenece la práctica (Ej: Ambiente, Calidad, Gestión, Social, Ética)."
  },
  {
    "name": "tema",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Tema específico dentro de la dimensión (Ej: Agua, Suelo, Residuos)."
  },
  {
    "name": "buena_practica",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Descripción de la buena práctica general."
  },
  {
    "name": "accion",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Acción específica que debe realizar el predio para cumplir con la buena práctica."
  },
  {
    "name": "medio_de_verificacion",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Descripción detallada de los medios o documentos necesarios para verificar el cumplimiento de la acción."
  },
  {
    "name": "link",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace URL a más información o recursos relacionados con la práctica."
  }
]

In [6]:
df_estandar = pd.read_excel("data/estandar_pp.xlsx")

In [ ]:
df_estandar["link"] = df_estandar.link.apply(lambda link: shorten_url(link, os.getenv("TINYURL_API_TOKEN")))

In [10]:
project_id = 'agro-extension-digital-npe'
destination_table = 'sandbox_rsolar.estandar_pp'

try:
    pandas_gbq.to_gbq(
        df_estandar,
        destination_table,
        project_id=project_id,
        if_exists='replace',
        table_schema=table_schema
    )
    print(f"DataFrame cargado exitosamente en {destination_table} con el esquema especificado.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

100%|██████████| 1/1 [00:00<00:00, 1919.59it/s]

DataFrame cargado exitosamente en sandbox_rsolar.estandar_pp con el esquema especificado.


In [6]:
table_schema = [
  {
    "name": "dimension",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Dimensión principal a la que pertenece el recurso (Ej: Ambiente, Calidad, Social)."
  },
  {
    "name": "tema",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Tema específico dentro de la dimensión (Ej: Agua, Suelo, Biodiversidad)."
  },
  {
    "name": "tipodetalle",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Tipo de recurso o detalle (Ej: Señalética, TDR, Registro, Guía, Curso)."
  },
  {
    "name": "detalle",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Descripción detallada o título del recurso."
  },
  {
    "name": "codigo",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Código(s) asociado(s) al recurso, puede ser una lista separada por comas."
  },
  {
    "name": "link_web",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace web principal al recurso en ciruelacertificada.cl."
  },
  {
    "name": "link_curso_chile_agricola",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace al curso correspondiente en chileagricola.cl (si aplica)."
  },
  {
    "name": "link_pdf",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace directo al archivo PDF del recurso (si aplica)."
  },
  {
    "name": "link_word",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace directo al archivo Word (.docx) del recurso (si aplica)."
  },
  {
    "name": "link_excel",
    "type": "STRING",
    "mode": "NULLABLE",
    "description": "Enlace directo al archivo Excel (.xlsx) del recurso (si aplica)."
  }
]

In [7]:
df_recursos = pd.read_excel("data/recursos_pp.xlsx")

In [8]:
df_recursos['codigo_list'] = df_recursos['codigo'].str.split(r'\s*,\s*')

In [9]:
df_unnested = df_recursos.explode('codigo_list')

In [10]:
df_unnested = df_unnested.drop(columns=["codigo"]).reset_index(drop=True)

In [11]:
df_unnested = df_unnested.rename(columns={'codigo_list': 'codigo'})
df_unnested['codigo'] = df_unnested['codigo'].str.strip()

In [12]:
project_id = 'agro-extension-digital-npe'
destination_table = 'sandbox_rsolar.recursos_pp'

try:
    pandas_gbq.to_gbq(
        df_unnested,
        destination_table,
        project_id=project_id,
        if_exists='replace',
        table_schema=table_schema
    )
    print(f"DataFrame cargado exitosamente en {destination_table} con el esquema especificado.")

except Exception as e:
    print(f"Ocurrió un error: {e}")

DataFrame cargado exitosamente en sandbox_rsolar.recursos_pp con el esquema especificado.
